# 서울 소방서 100곳 크롤링하기

In [56]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import pandas as pd
from tqdm.notebook import tqdm
from urllib.parse import quote
import requests

## 1.크롤링

In [ ]:
# 1. (크롤링) 셀레니움으로 크롤링을 해서 소방서명, 주소, 전화번호 데이터를 가져와서 데이터프레임을 만든 후,
#    데이터프레임에 새로운 컬럼 안전센터여부를 만들고 소방서는 0, 안전센터는 1로 채우세요. (40점)

### 1-1.사이트 불러오기

In [38]:
chromedriver = 'chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

In [39]:
site = 'https://www.nfa.go.kr'
url = site + '/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=keyword&myX=&myY=&searchKeyword=%EC%84%9C%EC%9A%B8'
driver.get(url)
soup = BeautifulSoup(driver.page_source,'html.parser')

### 1-2.페이지에서 원하는 부분가져오기

In [20]:
lis = soup.select('.stations-list>li')
len(lis)

10

In [21]:
#소방서명, 주소, 전화번호 가져오기
li = lis[0]
name = li.select_one('.title').get_text()
name

'동작소방서'

In [14]:
addr = li.select_one('address').get_text()
addr

'서울특별시 동작구 여의대방로16길 55(신대방동)'

In [16]:
tel = li.select_one('.tel').get_text()
tel

'02-847-1190'

In [22]:
lines = []
for li in lis:
    name = li.select_one('.title').get_text()
    addr = li.select_one('address').get_text()
    tel = li.select_one('.tel').get_text()
    lines.append({'소방서명':name,'주소':addr,'전화번호':tel})

df = pd.DataFrame(lines)
df


,소방서명,주소,전화번호
0,동작소방서,서울특별시 동작구 여의대방로16길 55(신대방동),02-847-1190
1,서대문소방서,서울특별시 서대문구 연희로 182(연희동),02-3144-1190
2,광진소방서,서울특별시 광진구 광나루로 480(구의동),02-457-0119
3,송파소방서,서울특별시 송파구 오금로51길 56(마천동),02-403-2119
4,양천소방서,서울특별시 양천구 목동서로 180(목동),02-2655-1119
5,은평소방서,서울특별시 은평구 통일로 962(진관동),02-355-0119
6,노원소방서,서울특별시 노원구 한글비석로 1길 8(하계동),02-977-0119
7,종로소방서,서울특별시 종로구 종로1길 28(수송동),02-735-6119
8,도봉소방서,서울특별시 도봉구 도봉로 666(방학동),02-3492-3437
9,용산소방서,서울특별시 용산구 한강대로 167(한강로2가),02-794-0119


### 1-3.모든페이지 데이터 가져오기

In [50]:
lines=[]
for page in tqdm(range(1,11)):
    url='https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=keyword&myX=&myY=&searchKeyword=%EC%84%9C%EC%9A%B8&pageIndex='+str(page)
    driver.get(url)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select('.stations-list>li')

    for li in lis:
        name = li.select_one('.title').get_text()
        addr = li.select_one('address').get_text()
        tel = li.select_one('.tel').get_text()
        lines.append({'소방서명':name,'주소':addr,'전화번호':tel})

df2 = pd.DataFrame(lines)
df2.head()


  0%|          | 0/10 [00:00<?, ?it/s]

,소방서명,주소,전화번호
0,동작소방서,서울특별시 동작구 여의대방로16길 55(신대방동),02-847-1190
1,서대문소방서,서울특별시 서대문구 연희로 182(연희동),02-3144-1190
2,광진소방서,서울특별시 광진구 광나루로 480(구의동),02-457-0119
3,송파소방서,서울특별시 송파구 오금로51길 56(마천동),02-403-2119
4,양천소방서,서울특별시 양천구 목동서로 180(목동),02-2655-1119


In [51]:
df2['안전센터여부'] = df2.소방서명.str.contains('안전센터').astype(int)
df2.tail()

,소방서명,주소,전화번호,안전센터여부
95,송파소방서-운동장-119 안전센터,서울특별시 송파구 올림픽로 25 (잠실동),02-2203-2380,1
96,송파소방서-잠실-119 안전센터,서울특별시 송파구 석촌호수로 151 (잠실동),02-422-0119,1
97,송파소방서-방이-119 안전센터,서울특별시 송파구 강동대로 286 (방이동),02-409-0059,1
98,송파소방서-거여-119 안전센터,서울특별시 송파구 마천로 329 (마천동),02-400-0119,1
99,송파소방서-마천-119 안전센터,서울특별시 송파구 오금로51길 56 (마천동),02-3401-2119,1


In [59]:
df2.to_csv('서울시소방서100곳.csv')

## 2.카카오 오픈 API

In [52]:
# 2. (OpenAPI) 앞에서 구한 주소를 가지고, 카카오 로컬 Open API를 이용하여 소방서의 위도, 경도 정보를 구한 후 
#    데이터프레임에 추가하세요. (30점)

In [55]:
with open('kakaoapikey.txt') as file:
    kakao_key = file.read()

local_url = "https://dapi.kakao.com/v2/local/search/address.json"
header = {'Authorization' : f'KakaoAK {kakao_key}'}
addr = df2


In [60]:
lng_list, lat_list = [], []
for i in addr.index:
    url = f'{local_url}?query={quote(addr.주소[i])}'
    result = requests.get(url, headers=header).json()
    try:
        lng_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:
        print(addr.소방서명[i],addr.주소[i])

은평소방서-수색-119 안전센터 서울특별시 은평구 수색로 294 (수색동)


In [ ]:
#오류코드수정
addr.주소[30] = '서울특별시 은평구 수색로 320'

In [67]:
lng_list, lat_list = [], []
for i in addr.index:
    url = f'{local_url}?query={quote(addr.주소[i])}'
    result = requests.get(url, headers=header).json()
    try:
        lng_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:
        print(addr.소방서명[i],addr.주소[i])

In [68]:
addr['위도'] = lat_list
addr['경도'] = lng_list
addr.head(3)

,소방서명,주소,전화번호,안전센터여부,위도,경도
0,동작소방서,서울특별시 동작구 여의대방로16길 55(신대방동),02-847-1190,0,37.494672,126.917719
1,서대문소방서,서울특별시 서대문구 연희로 182(연희동),02-3144-1190,0,37.573205,126.935996
2,광진소방서,서울특별시 광진구 광나루로 480(구의동),02-457-0119,0,37.544826,127.082779


## 3.지도시각화

In [69]:
# 3. (지도시각화) 앞에서 구한 정보를 바탕으로 서울 지도(Open Street Map)에
#    표시하되, 다음의 조건을 만족시키세요. (30점)
# 	- 툴팁에는 소방서명과 전화번호가 표시되어야 함
# 	- 팝업에는 소방서 주소가 가로로 표시되어야 함
# 	- 소방서와 안전센터의 아이콘이 다르게 표시되어야 함
# 	- 서울소재 소방서 위치라는 제목이 표시되어야 함

In [104]:
import folium 

In [108]:
df1 = addr[addr.안전센터여부==0]
df2 = addr[addr.안전센터여부==1]

In [109]:
map = folium.Map(location=[37.559868, 126.967109] , zoom_start=14)
for i in df1.index:
    folium.Marker(
        location=[df1.위도[i], df1.경도[i]],
        popup=folium.Popup(df1.주소[i], max_width=200),
        tooltip=[df1.소방서명[i],df1.전화번호[i]],
        icon=folium.Icon(color='red', icon='glyphicon glyphicon-bell')
    ).add_to(map)

for i in df2.index:
    folium.Marker(
        location=[df2.위도[i], df2.경도[i]],
        popup=folium.Popup(df2.주소[i], max_width=200),
        tooltip=[df2.소방서명[i],df2.전화번호[i]],
        icon=folium.Icon(color='blue', icon='glyphicon glyphicon-bell')
    ).add_to(map)

title_html = '<h3 align="center" style="font-size:20px">서울소재 소방서 위치</h3>'    
map.get_root().html.add_child(folium.Element(title_html))
map